In [1]:
# TODO:
# reproducible results driven by seed
# correlation of features output for two networks. Or even FC training from from to other with L1/L2 norm?
# start develop liveconv
# don't forger penalise for operation, not (just) weight. Add stats on operations
# why features' weights small but not zero? should I try depth-separable conv?
# manual livenet for cifar10, and then think on auto?

%load_ext autoreload
%autoreload 2
import torch
import torch.utils
import torch.utils.data
import torchsummary
import numpy as np
import gc
from livenet.utils import set_seed
import onnx
import livenet
device = "cuda"
#device = "cpu"
#torch.set_default_device(device)
from ai_libs.simple_log import LOG


In [2]:
batch_size = 256
from livenet.datasets import TransformDataset
test_x, test_y = livenet.datasets.get_cifar10_test()
test = torch.utils.data.TensorDataset(test_x, test_y)
test_aug = TransformDataset(test, livenet.datasets.cifar10_test_transform)
test_loader = torch.utils.data.DataLoader(test_aug, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=4, pin_memory=True)

train_x, train_y = livenet.datasets.get_cifar10_train()
train = torch.utils.data.TensorDataset(train_x, train_y)
train_aug = TransformDataset(train, livenet.datasets.cifar10_train_transform)
train_loader = torch.utils.data.DataLoader(train_aug, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=16, pin_memory=True, prefetch_factor=2)



In [ ]:

train_aug = TransformDataset(train, livenet.datasets.cifar10_train_transform)


In [ ]:
img = train_aug[4][0].numpy()
img = img.transpose(1, 2, 0)
from matplotlib import pyplot as plt
#img = livenet.datasets._elastic_transform(img, (-4, 0))
img = (img * 128 + 127).astype(np.uint8)

#plt.imsave("/home/spometun/img.png", img)
plt.imshow( img )

In [ ]:
# from cifar_arch import EffNet, ResNet9, ResNet9Small
from livenet.v2.cifar_arch import ResNet9SmallSmart

set_seed(2)

# network = EffNet(device)
# network = ResNet9Small(3, 10, device)
network = ResNet9SmallSmart(3, 10, device)
torchsummary.summary(network, (3, 32, 32), device=device)



In [ ]:
dummy_input = torch.zeros(size=(1, 3, 32, 32), device=device)
torch.onnx.export(network, dummy_input, "/home/spometun/temp/resnet9small.onnx")

In [ ]:
np.set_printoptions(precision=3, suppress=True)
ax_in = (0, 2, 3)
ax_out = (1, 2, 3)
ax = ax_out
vals = list(network.parameters())[32].data.cpu().numpy()
vals_ref = list(network.parameters())[32].data
out_max = np.max(vals, axis=ax)
out_min = np.min(vals, axis=ax)
sign = -out_min > out_max
out = np.max(np.abs(vals), axis=ax)
#out *= 1 - sign * 2
LOG(f"{np.min(vals):.2f} {np.max(vals):.2f}")
print(np.sum(np.abs(vals) < 0.01), np.sum(np.abs(vals) > 0.0))
#vals_ref[np.abs(vals) < 0.05] = 0.0
out

In [5]:
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, train_loader, criterion, optimizer)
trainer.adaptive_lr = True
optimizer.learning_rate = 0.01



Adam


In [6]:
network.train()
# trainer.optimizer.learning_rate = 0.01
network._alpha = 10 * 1e-6
trainer.step(2000)


KeyboardInterrupt: 

In [ ]:
gc.collect()
torch.cuda.empty_cache()

def _infer_epoch(network, loader):
    preds = []
    ys = []
    with torch.no_grad():
        for x, y in iter(loader):
            x = x.to(network.device)
            pred = network(x)
            pred = pred.cpu()
            preds.append(pred)
            ys.append(y.cpu())
    pred = torch.concatenate(preds)
    y = torch.concatenate(ys)
    return pred, y


network.train()
#train_pred, train_labels = _infer_epoch(network, train_loader)
test_pred, test_labels = _infer_epoch(network, test_loader)

network.eval()
test_pred, test_labels = _infer_epoch(network, test_loader)
train_pred, train_labels = _infer_epoch(network, train_loader)

def calc_accuracy(predictions, labels):
    _, predicted = torch.max(predictions.data, 1)
    labels = labels.numpy()
    labels = np.squeeze(labels, 1)
    predicted = predicted.numpy()
    correct = np.sum(predicted == labels)
    total = len(labels)
    accuracy = correct / total
    return accuracy


# test_y_tensor = torch.tensor(test_y, device="cpu")
# train_y_tensor = torch.tensor(train_y, device="cpu")
test_loss = trainer.criterion(test_pred, test_labels).item()
train_loss = trainer.criterion(train_pred, train_labels).item()
LOG(f"loss: train: {train_loss:.3f} test: {test_loss:.3f}")

test_accuracy = calc_accuracy(test_pred, test_labels)
train_accuracy = calc_accuracy(train_pred, train_labels)
LOG(f"accuracy, train: {100 * train_accuracy:.1f}% test: {100 * test_accuracy:.1f}%")

In [ ]:
network.to("cpu")

In [ ]:
param = list(network.named_parameters())
param0 = list(network.parameters())


In [ ]:
w = param[12][1].detach().cpu().numpy()
m = np.max(np.abs(w), axis=(1, 2, 3))
f"{int(100 * np.sum(m > 0.01) / len(m))}%"
np.sum(m > 0.01)

In [ ]:
outs = dict()
for i in range(11):
    ind = i * 4
    if ind >= 36:
        ind = 32 + (i - 8)
    w = param[ind][1].detach().cpu().numpy()
    w_name = param[ind][0]
    m_out = np.max(np.abs(w), axis=(1, 2, 3))
    outs[ind] = m_out > 0.01
    m_in = np.max(np.abs(w), axis=(0, 2, 3))
    n_out = np.sum(m_out > 0.01)
    n_in = np.sum(m_in > 0.01)
    s = f"{w_name} {ind}, {w.shape} {int(100 * np.sum(m_out > 0.01) / len(m_out))}%, {n_in}->{n_out}"
    print(s)
for ind in [12, 28]:
    ind2 = 32 if ind == 12 else 33
    n_inter = np.sum(outs[ind] * outs[ind2])
    print(f"{ind} inter={n_inter}")


In [ ]:
np.set_printoptions(1)
w = param[12][1].detach().cpu().numpy()
wa = np.abs(w)
wa_max = np.max(wa, axis=(1, 2, 3))
i = np.argsort(wa_max)[::-1]
f = w[i[0]]
fa = np.abs(f)
fa_max = np.max(fa, axis=(1, 2))
#print(f"{wa_max[i]}")
# print(wa[i].shape)
i1 = np.argsort(fa_max)[::-1]
f1b = fa > 0.1
np.sum(f1b)
f[i1[2]]
# fa[69]
#f[i1]